# Preparação dos Dados

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [4]:
caminho_arquivo = '/content/drive/MyDrive/JBCS/anotadores-sexismo - anotadores-sexismo.csv'
df = pd.read_csv(caminho_arquivo)

In [5]:
df.head()

,Id,Frase,Categoria 1,Categoria 2,Categoria 3,Categoria 4,Categoria 5,Sexismo,Resultado
0,1,ela é muito sensual,Sexismo Hostil,Sexismo Benevolente,Sexismo Hostil,Sexismo Hostil,Sexismo Hostil,Sexismo Hostil,Sexismo Hostil
1,2,menina bela,Não contém,Não contém,Não contém,Não contém,Não categorizado,Não contém,Não contém
2,4,ela é linda,Não contém,Não contém,Não contém,Não contém,Não contém,Não contém,Não contém
3,6,ela é mulher,Não contém,Não contém,Não contém,Não contém,Não contém,Não contém,Não contém
4,24,"mulher, menina",Não contém,Não contém,Indeterminado,Não contém,Não contém,Não contém,Não contém


In [6]:
df.shape

(2375, 9)

In [8]:
df = df[df['Resultado'].notna()]

In [7]:
categoria_counts = df['Resultado'].value_counts()
print(categoria_counts)

Resultado
Não contém             1622
Sexismo Hostil          547
Sexismo Benevolente     206
Name: count, dtype: int64


In [9]:
X = df['Frase'].values  # Frases
y = df['Resultado'].values  # Rótulos

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Resultado'])

In [12]:
classes_mapeadas = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print(classes_mapeadas)

{'Não contém': 0, 'Sexismo Benevolente': 1, 'Sexismo Hostil': 2}


In [13]:
from imblearn.over_sampling import RandomOverSampler

In [14]:
def balance_classes(X, y):
    ros = RandomOverSampler(random_state=42)
    X, y = ros.fit_resample(X.reshape(-1, 1), y)
    return X.flatten(), y

In [15]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW

In [16]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Treinamento

In [17]:
# treinamento
def train_model(model, data_loader, optimizer):
    model = model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    return total_loss / len(data_loader)

# Avaliação

In [18]:
# avaliação
from sklearn.metrics import classification_report

def evaluate_model(model, data_loader):
    model = model.eval()
    total_correct = 0
    total_samples = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)

            total_correct += torch.sum(preds == labels)
            total_samples += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    # Gera o classification report
    class_report = classification_report(all_labels, all_preds, target_names=label_encoder.classes_)

    return accuracy, precision, recall, f1, class_report, all_preds


In [22]:
# tokenizer e o modelo BERTimbau
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-large-portuguese-cased', num_labels=3)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-large-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Validação Cruzada Estratificada


In [23]:
# Definir o número de folds para validação cruzada
n_splits = 30 #10 é o mais comum, mas pode aumentar o número de folds desde que contenha no minimo 30 exemplos em cada fold, ex: 100 exemplos anotados, deve ter no maximo 3 folds / Deve-se saber tbm, que quanto maior o número de folds, maior o custo computacional
skf = StratifiedKFold(n_splits=n_splits)

In [24]:
# Otimizador
optimizer = AdamW(model.parameters(), lr=1e-5)

In [25]:
all_fold_metrics = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': []
}

In [26]:
all_true_labels = []
all_pred_labels = []

# Treinamento e Avaliação de Modelo com Validação Cruzada Estratificada e Balanceamento de Classes

In [27]:
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Fold {fold+1}/{n_splits}")

    # Dividir os dados em treino e teste
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Balancear as classes no conjunto de treino
    X_train_balanced, y_train_balanced = balance_classes(X_train, y_train)

    # Criar datasets e dataloaders
    train_dataset = TextDataset(X_train_balanced, y_train_balanced, tokenizer)
    test_dataset = TextDataset(X_test, y_test, tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)  # mini-batch de tamanho 16 e shuffle para pegar amostras aleatórias
    test_loader = DataLoader(test_dataset, batch_size=16) # mini-batch de tamanho 16

    # Treinar o modelo no fold atual
    train_loss = train_model(model, train_loader, optimizer)
    print(f"Loss de treino no Fold {fold+1}: {train_loss}") #quanto menor a loss em cada fold significa que o modelo está aprendendo

    # Avaliar o modelo no fold atual
    accuracy, precision, recall, f1, class_report, all_preds = evaluate_model(model, test_loader)
    #print(f"Acurácia no Fold {fold+1}: {accuracy}")
    #print(f"Precisão no Fold {fold+1}: {precision}")
    #print(f"Recall no Fold {fold+1}: {recall}")
    #print(f"F1-score no Fold {fold+1}: {f1}")
    #print(f"Classification Report no Fold {fold+1}:\n{class_report}")

    # Armazenar métricas para cada fold
    all_fold_metrics['accuracy'].append(accuracy)
    all_fold_metrics['precision'].append(precision)
    all_fold_metrics['recall'].append(recall)
    all_fold_metrics['f1'].append(f1)

    all_true_labels.extend(y_test)
    all_pred_labels.extend(all_preds)

# Cálculo das métricas finais (médias)
final_accuracy = np.mean(all_fold_metrics['accuracy'])
final_precision = np.mean(all_fold_metrics['precision'])
final_recall = np.mean(all_fold_metrics['recall'])
final_f1 = np.mean(all_fold_metrics['f1'])


print(f"\nDesempenho Final:")
print(f"Accuracy Média: {final_accuracy}")
print(f"Precision Média: {final_precision}")
print(f"Recall Médio: {final_recall}")
print(f"F1-score Médio: {final_f1}")

# Geração do Classification Report geral
final_class_report = classification_report(all_true_labels, all_pred_labels, target_names=label_encoder.classes_)
print("\nClassification Report:")
print(final_class_report)

Fold 1/30
Loss de treino no Fold 1: 0.6380189815852918
Fold 2/30
Loss de treino no Fold 2: 0.2229721798833941
Fold 3/30
Loss de treino no Fold 3: 0.11690106511204726
Fold 4/30
Loss de treino no Fold 4: 0.07814702227375894
Fold 5/30
Loss de treino no Fold 5: 0.05450211503055124
Fold 6/30
Loss de treino no Fold 6: 0.03209259005249193
Fold 7/30
Loss de treino no Fold 7: 0.028713959111233374
Fold 8/30
Loss de treino no Fold 8: 0.03607003617996895
Fold 9/30
Loss de treino no Fold 9: 0.023993835112532967
Fold 10/30
Loss de treino no Fold 10: 0.010323426828083748
Fold 11/30
Loss de treino no Fold 11: 0.02007796473604735
Fold 12/30
Loss de treino no Fold 12: 0.015077100140236629
Fold 13/30
Loss de treino no Fold 13: 0.010232534060978528
Fold 14/30
Loss de treino no Fold 14: 0.011168794082909297
Fold 15/30
Loss de treino no Fold 15: 0.01348560223948657
Fold 16/30
Loss de treino no Fold 16: 0.007842261226640675
Fold 17/30
Loss de treino no Fold 17: 0.016808595705975787
Fold 18/30
Loss de treino 

Baixar modelo para uso posterior

In [ ]:
!mkdir model
model.save_pretrained("./model")
tokenizer.save_pretrained("./model")

!zip -r /content/model.zip /content/model

from google.colab import files
files.download("/content/model.zip")

mkdir: cannot create directory ‘model’: File exists
updating: content/model/ (stored 0%)
updating: content/model/model.safetensors (deflated 7%)
updating: content/model/vocab.txt (deflated 45%)
updating: content/model/tokenizer_config.json (deflated 75%)
updating: content/model/special_tokens_map.json (deflated 42%)
updating: content/model/config.json (deflated 55%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>